In [1]:
import yaml
import os
import sys
import json
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


## Prepare a txt and dict of all var and query_type and associated paraphrases

### Vars

In [9]:

# Get list of all yml files in ../../cladder/assets/stories/
path = '../../cladder/assets/stories/'
files = os.listdir(path)
files_yml = [f for f in files if f[-4:] == '.yml']
# sort files_yml
files_yml.sort()
print(files_yml)

['alarm.yml', 'blood_pressure.yml', 'candle.yml', 'celebrity.yml', 'cholesterol.yml', 'college_salary.yml', 'college_wage.yml', 'elite_students.yml', 'encouagement_program.yml', 'firing_employee.yml', 'firing_squad.yml', 'floor_wet.yml', 'forest_fire.yml', 'gender_admission.yml', 'gender_admission_state.yml', 'gender_pay.yml', 'getting_late.yml', 'getting_tanned.yml', 'hospitalization.yml', 'man_in_relationship.yml', 'nature_vs_nurture.yml', 'neg_mediation.yml', 'nonsense0.yml', 'nonsense1.yml', 'nonsense2.yml', 'nonsense3.yml', 'nonsense4.yml', 'nonsense5.yml', 'nonsense6.yml', 'nonsense7.yml', 'nonsense8.yml', 'nonsense9.yml', 'obesity_mortality.yml', 'orange_scurvy.yml', 'penguin.yml', 'price.yml', 'simpson_drug.yml', 'simpson_hospital.yml', 'simpson_kidneystone.yml', 'simpson_vaccine.yml', 'smoke_birthWeight.yml', 'smoking_frontdoor.yml', 'smoking_gene_cancer.yml', 'smoking_tar_cancer.yml', 'tax_smoke_birthWeight.yml', 'vaccine_kills.yml', 'water_cholera.yml']


In [13]:
# Extract Xname, X1_noun, Yname, Y1_noun from each yml file
df = pd.DataFrame(columns=['Xname', 'X1_noun', 'Yname', 'Y1_noun'])
for file in files_yml:
    with open(path + file) as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], 'Yname': data['Yname'], 'Y1_noun': data['Y1_noun']}, ignore_index=True)

df.head()

/var/folders/4l/50kz2_m54d53hffxvvphv1lr0000gn/T/ipykernel_73844/2599462131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], 'Yname': data['Yname'], 'Y1_noun': data['Y1_noun']}, ignore_index=True)
/var/folders/4l/50kz2_m54d53hffxvvphv1lr0000gn/T/ipykernel_73844/2599462131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], 'Yname': data['Yname'], 'Y1_noun': data['Y1_noun']}, ignore_index=True)
/var/folders/4l/50kz2_m54d53hffxvvphv1lr0000gn/T/ipykernel_73844/2599462131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], '

47


/var/folders/4l/50kz2_m54d53hffxvvphv1lr0000gn/T/ipykernel_73844/2599462131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], 'Yname': data['Yname'], 'Y1_noun': data['Y1_noun']}, ignore_index=True)
/var/folders/4l/50kz2_m54d53hffxvvphv1lr0000gn/T/ipykernel_73844/2599462131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], 'Yname': data['Yname'], 'Y1_noun': data['Y1_noun']}, ignore_index=True)
/var/folders/4l/50kz2_m54d53hffxvvphv1lr0000gn/T/ipykernel_73844/2599462131.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Xname': data['Xname'], 'X1_noun': data['X1_noun'], '

,Xname,X1_noun,Yname,Y1_noun
0,husband,alarm set by husband,alarm clock,ringing alarm
1,medication,taking medication,heart condition,healthy heart
2,the man in the room,blowing out the candle,room,dark room
3,appearance,attractive appearance,talent,talent
4,drug taken,taking of all assigned drugs,cholesterol level,low cholesterol


In [15]:
# Check if Xname and X1_noun are the same in some cases
df[df['Xname'] == df['X1_noun']]

,Xname,X1_noun,Yname,Y1_noun
7,talent,talent,effort,being hard-working
17,tanning salon treatment,tanning salon treatment,skin,tanned skin
18,respiratory issues,respiratory issues,broken bones,broken bones
21,smoking,smoking,college admission,college admission
22,rixq,rixq,xevu,xevu
23,kwox,kwox,kwoz,kwoz
24,xevo,xevo,gyzp,gyzp
25,yomx,yomx,xevu,xevu
26,jyka,jyka,lirg,lirg
27,jyka,jyka,kwox,kwox


In [48]:
# create a dictionary with Xname as key and Xname as value, if Xname and X1_noun are different also append X1_noun as a value to the key Xname

Xname_dict = {}
Yname_dict = {}
for index, row in df.iterrows():
    if row['Xname'] == row['X1_noun']:
        Xname_dict[row['Xname']] = row['Xname']
    else:
        Xname_dict[row['Xname']] = [row['Xname'], row['X1_noun']]
    if row['Yname'] == row['Y1_noun']:
        Yname_dict[row['Yname']] = row['Yname']
    else:
        Yname_dict[row['Yname']] = [row['Yname'], row['Y1_noun']]

# merge Xname_dict and Yname_dict
var_dict = {**Xname_dict, **Yname_dict}
var_dict

{'husband': ['husband', 'alarm set by husband'],
 'medication': ['medication', 'taking medication'],
 'the man in the room': ['the man in the room', 'blowing out the candle'],
 'appearance': ['appearance', 'attractive appearance'],
 'drug taken': ['drug taken', 'taking of all assigned drugs'],
 'education level': ['education level', 'college degree or higher'],
 'talent': 'talent',
 'encouragement level': ['encouragement level', 'encouragement'],
 'manager': ['manager', 'manager signing the termination letter'],
 'the captain': ['the captain', "the captain's order to execute the prisoner"],
 'season': ['season', 'rainy season'],
 'the camper': ['the camper', 'the camper lighting a match'],
 'gender': ['gender', 'male gender'],
 'Alice waking up': ['Alice waking up', 'waking up late'],
 'tanning salon treatment': 'tanning salon treatment',
 'respiratory issues': 'respiratory issues',
 'personality': ['personality', 'kindness'],
 "parents' intelligence": ["parents' intelligence", 'intell

In [22]:
# Write a txt file, each line is one element of Xname or X1_noun or Yname or Y1_noun

# Get list of all Xname, X1_noun, Yname, Y1_noun
import dis


Xname_list = df['Xname'].tolist()
X1_noun_list = df['X1_noun'].tolist()
Yname_list = df['Yname'].tolist()
Y1_noun_list = df['Y1_noun'].tolist()

# Combine all lists
var_list = Xname_list
var_list.extend(X1_noun_list)
var_list.extend(Yname_list)
var_list.extend(Y1_noun_list)

# Remove duplicates
var_list = list(dict.fromkeys(var_list))
var_list

['husband',
 'medication',
 'the man in the room',
 'appearance',
 'drug taken',
 'education level',
 'talent',
 'encouragement level',
 'manager',
 'the captain',
 'season',
 'the camper',
 'gender',
 'Alice waking up',
 'tanning salon treatment',
 'respiratory issues',
 'personality',
 "parents' intelligence",
 'smoking',
 'rixq',
 'kwox',
 'xevo',
 'yomx',
 'jyka',
 'yupt',
 'zuph',
 'pexu',
 'obesity',
 'eating citrus',
 'my decision',
 'supply',
 'treatment',
 'hospital costs',
 'vaccination',
 'maternal smoking status',
 'gene',
 'vaccination status',
 'water quality',
 'alarm set by husband',
 'taking medication',
 'blowing out the candle',
 'attractive appearance',
 'taking of all assigned drugs',
 'college degree or higher',
 'encouragement',
 'manager signing the termination letter',
 "the captain's order to execute the prisoner",
 'rainy season',
 'the camper lighting a match',
 'male gender',
 'waking up late',
 'kindness',
 'intelligent parents',
 'citrus intake',
 'taking

### Query types

In [52]:
query_type_string = """ATE, Average treatment effect, ACE, average causal effect
ATT, Average Treatment Effect on the Treated, Effect of the Treatment on the Treated, ETT
NDE, Natural direct effect
NIE, Natural indirect effect
Correlation
Marginal Distribution, marginal
Expaining Away Effect, exp_away
Backdoor Adjustment Set, backadj
Collider Bias, collider_bias
Counterfactual (deterministic), det-counterfactua"""

In [54]:
# Parse query_type to df: split by \n, then by , first element in one column all others in another column
query_type = query_type_string.split('\n')
query_type = [q.split(',') for q in query_type]
#query_type = pd.DataFrame(query_type, columns=['query_type', 'query_type_short'])
#query_type = query_type[query_type['query_type'] != '']
query_type
# For each list in query_type, remove blank spaces at the beginning and end of each element
query_type = [[q.strip() for q in l] for l in query_type]
query_type

[['ATE', 'Average treatment effect', 'ACE', 'average causal effect'],
 ['ATT',
  'Average Treatment Effect on the Treated',
  'Effect of the Treatment on the Treated',
  'ETT'],
 ['NDE', 'Natural direct effect'],
 ['NIE', 'Natural indirect effect'],
 ['Correlation'],
 ['Marginal Distribution', 'marginal'],
 ['Expaining Away Effect', 'exp_away'],
 ['Backdoor Adjustment Set', 'backadj'],
 ['Collider Bias', 'collider_bias'],
 ['Counterfactual (deterministic)', 'det-counterfactua']]

In [41]:
# Create a dictionary with first element as key, all other elements as values + the key repeated
query_type_dict = {}
for l in query_type:
    query_type_dict[l[0]] = l[1:]
    query_type_dict[l[0]].append(l[0])
query_type_dict

{'ATE': ['Average treatment effect', 'ACE', 'average causal effect', 'ATE'],
 'ATT': ['Average Treatment Effect on the Treated',
  'Effect of the Treatment on the Treated',
  'ETT',
  'ATT'],
 'NDE': ['Natural direct effect', 'NDE'],
 'NIE': ['Natural indirect effect', 'NIE'],
 'Correlation': ['Correlation'],
 'Marginal Distribution': ['marginal', 'Marginal Distribution'],
 'Expaining Away Effect': ['exp_away', 'Expaining Away Effect'],
 'Backdoor Adjustment Set': ['backadj', 'Backdoor Adjustment Set'],
 'Collider Bias': ['collider_bias', 'Collider Bias'],
 'Counterfactual (deterministic)': ['det-counterfactua',
  'Counterfactual (deterministic)']}

In [56]:
# Create a list of all elements in query_type
query_type_list = []
for l in query_type:
    query_type_list.extend(l)
query_type_list


['ATE',
 'Average treatment effect',
 'ACE',
 'average causal effect',
 'ATT',
 'Average Treatment Effect on the Treated',
 'Effect of the Treatment on the Treated',
 'ETT',
 'NDE',
 'Natural direct effect',
 'NIE',
 'Natural indirect effect',
 'Correlation',
 'Marginal Distribution',
 'marginal',
 'Expaining Away Effect',
 'exp_away',
 'Backdoor Adjustment Set',
 'backadj',
 'Collider Bias',
 'collider_bias',
 'Counterfactual (deterministic)',
 'det-counterfactua']

### Combine vars and query_type

In [60]:
# Combine var_list and query_type_list
combined_list = var_list
combined_list.extend(query_type_list)

# Combine var_dict and query_type_dict
combined_dict = {**var_dict, **query_type_dict}

In [61]:
# Write to txt file
with open('../data/pile_query.txt', 'w') as f:
    for item in combined_list:
        f.write("%s\n" % item)